In [1]:
#Importing the necessary libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


In [4]:
# Load the data
df = pd.read_csv('credit_risk_dataset.csv') 

# Select features and target
features = ['person_age', 'person_income', 'person_home_ownership', 'person_emp_length',
            'loan_intent', 'loan_amnt', 'loan_int_rate', 'cb_person_default_on_file',
            'cb_person_cred_hist_length']
target = 'loan_status'

X = df[features]
y = df[target]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the first 5 rows of the dataset
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [5]:
# Create preprocessing steps
numeric_features = ['person_age', 'person_income', 'person_emp_length', 'loan_amnt', 
                    'loan_int_rate', 'cb_person_cred_hist_length']
categorical_features = ['person_home_ownership', 'loan_intent', 'cb_person_default_on_file']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ])

In [7]:
#Define the CreditRiskAgent class

class CreditRiskAgent:
    def __init__(self):
        self.model = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
        ])

    def train(self, X, y):
        self.model.fit(X, y)

    def predict_risk(self, features):
        return self.model.predict_proba(features)[0][1]  # Probability of high risk

    def make_decision(self, risk_probability):
        if risk_probability < 0.3:
            return "Low Risk - Approve Loan"
        elif risk_probability < 0.7:
            return "Medium Risk - Further Review Required"
        else:
            return "High Risk - Deny Loan"

In [9]:
# Apply the CoT reasoning to the CreditRiskAgent by inheriting the class 

class CoTCreditRiskAgent(CreditRiskAgent):
    def __init__(self):
        super().__init__()
        self.thought_process = [
            self._assess_personal_info,
            self._evaluate_financial_status,
            self._analyze_loan_details,
            self._consider_credit_history,
            self._calculate_overall_risk
        ]

    def assess_application(self, applicant_data):
        features = pd.DataFrame([applicant_data])
        risk_probability = self.predict_risk(features)
        
        thoughts = []
        for step in self.thought_process:
            thought = step(applicant_data, risk_probability)
            thoughts.append(thought)
        
        final_decision = self.make_decision(risk_probability)
        return final_decision, thoughts

    def _assess_personal_info(self, data, risk_prob):
        age = data['person_age']
        emp_length = data['person_emp_length']
        assessment = f"Applicant age: {age} years. Employment length: {emp_length} years. "
        if age < 25:
            assessment += "Young applicant. "
        elif age > 60:
            assessment += "Senior applicant. "
        if emp_length < 2:
            assessment += "Limited employment history."
        elif emp_length > 10:
            assessment += "Stable employment history."
        return assessment

    def _evaluate_financial_status(self, data, risk_prob):
        income = data['person_income']
        home_ownership = data['person_home_ownership']
        assessment = f"Annual income: ${income:,}. Home ownership: {home_ownership}. "
        if income < 30000:
            assessment += "Low income. "
        elif income > 100000:
            assessment += "High income. "
        if home_ownership == 'OWN':
            assessment += "Owns home, potential asset."
        elif home_ownership == 'RENT':
            assessment += "Renting, no property asset."
        return assessment

    def _analyze_loan_details(self, data, risk_prob):
        loan_amount = data['loan_amnt']
        loan_intent = data['loan_intent']
        int_rate = data['loan_int_rate']
        return f"Loan amount: ${loan_amount:,}. Purpose: {loan_intent}. Interest rate: {int_rate}%. " + \
               ("High interest rate. " if int_rate > 15 else "Reasonable interest rate. ")

    def _consider_credit_history(self, data, risk_prob):
        default_history = data['cb_person_default_on_file']
        credit_length = data['cb_person_cred_hist_length']
        assessment = f"Default on file: {default_history}. Credit history length: {credit_length} years. "
        if default_history == 'Y':
            assessment += "Previous default is concerning. "
        else:
            assessment += "No defaults on record. "
        if credit_length < 5:
            assessment += "Limited credit history."
        elif credit_length > 10:
            assessment += "Extensive credit history."
        return assessment

    def _calculate_overall_risk(self, data, risk_prob):
        return f"Calculated risk probability: {risk_prob:.2f}. " + \
               ("High risk." if risk_prob > 0.6 else 
                "Moderate risk." if risk_prob > 0.3 else "Low risk.")

In [11]:
class RLCoTCreditRiskAgent(CoTCreditRiskAgent):
    def __init__(self):
        super().__init__()
        self.q_table = {}
        self.learning_rate = 0.1
        self.discount_factor = 0.9
        self.epsilon = 0.1

    def get_state(self, features):
        # Simplify the state representation for Q-learning
        return (
            round(features['person_age'] / 10) * 10,
            round(features['person_income'] / 10000) * 10000,
            features['person_home_ownership'],
            round(features['person_emp_length']),
            features['loan_intent'],
            round(features['loan_amnt'] / 5000) * 5000,
            round(features['loan_int_rate']),
            features['cb_person_default_on_file'],
            round(features['cb_person_cred_hist_length'] / 5) * 5
        )

    def choose_action(self, state):
        if np.random.random() < self.epsilon:
            return np.random.choice(['approve', 'deny'])
        
        if state not in self.q_table:
            self.q_table[state] = {'approve': 0, 'deny': 0}
        
        return max(self.q_table[state], key=self.q_table[state].get)

    def update_q_table(self, state, action, reward, next_state):
        if state not in self.q_table:
            self.q_table[state] = {'approve': 0, 'deny': 0}
        
        if next_state not in self.q_table:
            self.q_table[next_state] = {'approve': 0, 'deny': 0}
        
        current_q = self.q_table[state][action]
        max_next_q = max(self.q_table[next_state].values())
        
        new_q = (1 - self.learning_rate) * current_q + \
                self.learning_rate * (reward + self.discount_factor * max_next_q)
        
        self.q_table[state][action] = new_q

    def assess_application_with_rl(self, applicant_data):
        features = pd.DataFrame([applicant_data])
        state = self.get_state(applicant_data)
        
        rl_action = self.choose_action(state)
        model_decision, thoughts = self.assess_application(applicant_data)
        
        final_decision = 'approve' if rl_action == 'approve' and 'Approve' in model_decision else 'deny'
        
        return final_decision, thoughts, rl_action

    def learn_from_outcome(self, applicant_data, action, reward):
        state = self.get_state(applicant_data)
        next_state = state  # In this case, the state doesn't change
        self.update_q_table(state, action, reward, next_state)

In [12]:
# Create and train the agents
cot_agent = CoTCreditRiskAgent()
cot_agent.train(X_train, y_train)

rl_cot_agent = RLCoTCreditRiskAgent()
rl_cot_agent.train(X_train, y_train)

# Example application
example_applicant = {
    'person_age': 30,
    'person_income': 60000,
    'person_home_ownership': 'RENT',
    'person_emp_length': 5,
    'loan_intent': 'PERSONAL',
    'loan_amnt': 10000,
    'loan_int_rate': 10.0,
    'cb_person_default_on_file': 'N',
    'cb_person_cred_hist_length': 5
}

# Assess with CoT agent
cot_decision, cot_thoughts = cot_agent.assess_application(example_applicant)
print("CoT Agent Decision:", cot_decision)
print("CoT Agent Thoughts:")
for thought in cot_thoughts:
    print("-", thought)

# Assess with RL-CoT agent
rl_decision, rl_thoughts, rl_action = rl_cot_agent.assess_application_with_rl(example_applicant)
print("\nRL-CoT Agent Decision:", rl_decision)
print("RL-CoT Agent Action:", rl_action)
print("RL-CoT Agent Thoughts:")
for thought in rl_thoughts:
    print("-", thought)

CoT Agent Decision: Low Risk - Approve Loan
CoT Agent Thoughts:
- Applicant age: 30 years. Employment length: 5 years. 
- Annual income: $60,000. Home ownership: RENT. Renting, no property asset.
- Loan amount: $10,000. Purpose: PERSONAL. Interest rate: 10.0%. Reasonable interest rate. 
- Default on file: N. Credit history length: 5 years. No defaults on record. 
- Calculated risk probability: 0.05. Low risk.

RL-CoT Agent Decision: approve
RL-CoT Agent Action: approve
RL-CoT Agent Thoughts:
- Applicant age: 30 years. Employment length: 5 years. 
- Annual income: $60,000. Home ownership: RENT. Renting, no property asset.
- Loan amount: $10,000. Purpose: PERSONAL. Interest rate: 10.0%. Reasonable interest rate. 
- Default on file: N. Credit history length: 5 years. No defaults on record. 
- Calculated risk probability: 0.05. Low risk.


In [16]:
# First, let's train and evaluate the base CreditRiskAgent
base_agent = RLCoTCreditRiskAgent()
base_agent.train(X_train, y_train)

# Make predictions on the test set
y_pred_base = base_agent.model.predict(X_test)

# Calculate accuracy
accuracy_base = accuracy_score(y_test, y_pred_base)

# Generate classification report
report_base = classification_report(y_test, y_pred_base)

print("RL-Based CreditRiskAgent Performance:")
print(f"Accuracy: {accuracy_base:.4f}")
print("\nClassification Report:")
print(report_base)

RL-Based CreditRiskAgent Performance:
Accuracy: 0.9090

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5072
           1       0.90      0.67      0.76      1445

    accuracy                           0.91      6517
   macro avg       0.90      0.82      0.85      6517
weighted avg       0.91      0.91      0.90      6517

